In [ ]:
import os


os.chdir("../")


os.getcwd()


'd:\\DA_workspace\\car_price_prediction\\src'

In [ ]:
import pandas as pd
import numpy as np
from regressor.Mylib import myfuncs
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.feature_selection import (
    SelectKBest,
    mutual_info_regression,
    mutual_info_classif,
)


In [ ]:
train_data = myfuncs.load_python_object("../artifacts/data_correction/train_data.pkl")
val_data = myfuncs.load_python_object("../artifacts/data_correction/val_data.pkl")


df = pd.concat([train_data, val_data], axis=0).reset_index(drop=True)

df.head()


,Year_num,Engine_Size_num,Mileage_num,Doors_count_num,old_Owner_Count_num,Brand_nom,Model_nom,Fuel_Type_nom,Transmission_type_nom,Price
0,2003.0,1.5,28708.0,3.0,4.0,Toyota,Corolla,Electric,Semi-Automatic,7925.0
1,2019.0,3.0,252522.0,3.0,4.0,Chevrolet,Malibu,Petrol,Manual,7749.0
2,2005.0,1.1,171245.0,4.0,4.0,Ford,Focus,Petrol,Semi-Automatic,3275.0
3,2003.0,1.2,286130.0,2.0,1.0,Honda,CR-V,Petrol,Manual,2000.0
4,2012.0,3.3,253002.0,3.0,3.0,Kia,Rio,Hybrid,Automatic,8439.0


In [ ]:
df.columns


Index(['Year_num', 'Engine_Size_num', 'Mileage_num', 'Doors_count_num',
       'old_Owner_Count_num', 'Brand_nom', 'Model_nom', 'Fuel_Type_nom',
       'Transmission_type_nom', 'Price'],
      dtype='object')

In [ ]:
cols = pd.Series(df.columns)
numeric_cols = cols[cols.str.endswith("num")].tolist()

numeric_cat_cols = [
    "Year_num",
    "Doors_count_num",
    "old_Owner_Count_num",
]

numeric_cols = list(set(numeric_cols) - set(numeric_cat_cols))

nominal_cols = cols[cols.str.endswith("nom")].tolist()

cat_cols = nominal_cols + numeric_cat_cols

target_col = "Price"


Cac cot numeric hien thi histogram, box plot và violinplot


In [ ]:
df["Engine_Size_num"].unique()


array([1.5, 3. , 1.1, 1.2, 3.3, 3.8, 4.2, 4.3, 1.4, 2.9, 3.9, 2.4, 1.3,
       4.9, 3.5, 3.4, 3.6, 1.8, 1.6, 4.6, 3.1, 4.4, 2.5, 3.2, 1.7, 2.1,
       4. , 4.8, 2.7, 2. , 4.1, 2.6, 4.5, 3.7, 4.7, 2.8, 5. , 1.9, 1. ,
       2.2, 2.3])

In [ ]:
subplot_titles = sum(([item, "", ""] for item in numeric_cols), [])


fig = make_subplots(rows=len(numeric_cols), cols=3, subplot_titles=subplot_titles)

for row, col in enumerate(numeric_cols, 1):
    data = df[col]

    # Vẽ Histogram
    fig_hist = px.histogram(
        x=data,  # Dữ liệu cho trục X
        nbins=30,  # Số lượng cột (bins)
    )

    # Thêm đường viền cho các cột histogram
    fig_hist.update_traces(marker=dict(line=dict(width=1, color="black")))

    # Vẽ box plot
    fig_box = px.box(
        y=data,  # Dữ liệu trục Y
    )

    # Vẽ violin plot
    fig_violin = px.violin(
        y=data,  # Dữ liệu trên trục Y
        box=True,  # Hiển thị Box Plot bên trong
    )

    fig.add_trace(fig_hist.data[0], row=row, col=1)
    fig.add_trace(fig_box.data[0], row=row, col=2)
    fig.add_trace(fig_violin.data[0], row=row, col=3)

fig.update_layout(
    width=400 * 3,  # Độ rộng biểu đồ (px)
    height=len(numeric_cols) * 500,  # Độ cao biểu đồ (px)
)

fig.show()


Cac cot numeric

- Cac cot co phan bo histogram kha dong deu


đối với các cột cat thì hiển thị tỉ lệ chiếm trong các cột đó là được rồi


In [ ]:
fig = make_subplots(rows=len(cat_cols), cols=1, subplot_titles=cat_cols)

for row, col in enumerate(cat_cols, 1):
    data = df[col]

    percent = data.value_counts() / len(data) * 100

    # Vẽ Barh chart the hien ti le cua tung label
    fig_bar = px.bar(
        x=percent.values,  # Giá trị trục hoành
        y=percent.index,  # Danh mục trục tung
        orientation="h",  # Cột ngang
    )

    fig.add_trace(fig_bar.data[0], row=row, col=1)

fig.update_layout(
    width=800,  # Độ rộng biểu đồ (px)
    height=len(cat_cols) * 500,  # Độ cao biểu đồ (px)
)

fig.show()


In [ ]:
df.columns


Index(['Year_num', 'Engine_Size_num', 'Mileage_num', 'Doors_count_num',
       'old_Owner_Count_num', 'Brand_nom', 'Model_nom', 'Fuel_Type_nom',
       'Transmission_type_nom', 'Price'],
      dtype='object')

- Cac cot deu co ti le giua cac label deu


Phan bo cot Target


In [ ]:
subplot_titles = sum(([item, "", ""] for item in [target_col]), [])


fig = make_subplots(rows=len([target_col]), cols=3, subplot_titles=subplot_titles)

for row, col in enumerate([target_col], 1):
    data = df[col]

    # Vẽ Histogram
    fig_hist = px.histogram(
        x=data,  # Dữ liệu cho trục X
        nbins=30,  # Số lượng cột (bins)
    )

    # Thêm đường viền cho các cột histogram
    fig_hist.update_traces(marker=dict(line=dict(width=1, color="black")))

    # Vẽ box plot
    fig_box = px.box(
        y=data,  # Dữ liệu trục Y
    )

    # Vẽ violin plot
    fig_violin = px.violin(
        y=data,  # Dữ liệu trên trục Y
        box=True,  # Hiển thị Box Plot bên trong
    )

    fig.add_trace(fig_hist.data[0], row=row, col=1)
    fig.add_trace(fig_box.data[0], row=row, col=2)
    fig.add_trace(fig_violin.data[0], row=row, col=3)

fig.update_layout(
    width=400 * 3,  # Độ rộng biểu đồ (px)
    height=len([target_col]) * 500,  # Độ cao biểu đồ (px)
)

fig.show()


In [ ]:
df[target_col].describe()


count     9000.000000
mean      8850.021222
std       3122.099890
min       2000.000000
25%       6633.750000
50%       8853.000000
75%      11102.750000
max      18301.000000
Name: Price, dtype: float64